In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se
import re

In [3]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [4]:
# df = pd.read_csv('../../data/annotation_100_input_241201.csv')

In [5]:
df_one = pd.read_csv('../../data/part_one_annotation_50_input.csv')
df_two = pd.read_csv('../../data/part_two_annotation_50_input.csv')

In [6]:
# df_one.head(10)

In [7]:
df_one = df_one[~(df_one['Q_difficulty']=='-') & ~(df_one['A_difficulty']=='-') ].reset_index(drop=True)
df_two = df_two[~(df_two['Q_difficulty']=='-') & ~(df_two['A_difficulty']=='-') ].reset_index(drop=True)

In [8]:
df = pd.concat([df_one, df_two ], axis = 0)

In [9]:
# df = df[~(df['Q_difficulty']=='-') & ~(df['A_difficulty']=='-') ].reset_index(drop=True)

In [10]:
df.head()

,id,link,Q_difficulty,A_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate,Intermediate


In [11]:
q_df = df[['id', 'link', 'Q_difficulty']]
a_df = df[['id', 'link', 'A_difficulty']]

In [12]:
q_df.head()

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate


In [13]:
q_df

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate
...,...,...,...
28,74793352,https://stackoverflow.com/questions/74793352,Basic
29,74727945,https://stackoverflow.com/questions/74727945,Intermediate
30,74700764,https://stackoverflow.com/questions/74700764,Basic
31,77538219,https://stackoverflow.com/questions/77538219,Basic


In [14]:
q_id_str = str([x for x in q_df['id']])[1:-1]
a_id_str = str([x for x in a_df['id']])[1:-1]

In [15]:
q_id_str

'77293473, 77404924, 77249095, 77076663, 77019129, 76992629, 77370805, 76784524, 76608783, 76697269, 76317584, 76570219, 76302571, 76548866, 76541602, 76264395, 76169010, 76169378, 76059076, 76045819, 76085427, 75779691, 75862649, 75637544, 75460677, 75454689, 75283999, 75279786, 75198212, 75057003, 75179892, 75239843, 74778102, 74774389, 74760874, 74751254, 74693871, 74844950, 74804354, 77553477, 77543317, 77511053, 77467788, 77155599, 77291783, 77150535, 77068633, 77020489, 76917593, 76798081, 76774917, 76675433, 76427264, 76331128, 76367594, 76481433, 76115814, 76040179, 75870881, 75995957, 75711831, 75789316, 75942768, 75492157, 75352739, 75333570, 75266730, 75243277, 75022065, 74936129, 75099305, 74793352, 74727945, 74700764, 77538219, 77464645'

In [16]:
q_postypeid="'1'"
a_postypeid="'2'"



In [17]:
# q_sql = """select a.id 
#                 , 'Here is the question about software development. The title of the question is "'||a.title||'". and body of the question is "'||b.body||'". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question'
#             from posts a 
#                , postsbody b
#             where a.id = b.id
#               and a.creationdate between '2021-11-30' and '2023-11-30' 
#               and a.posttypeid = {0} 
#               and a.tags like '%<python>%' 
#               and a.id in ({1})
# """

In [18]:
q_sql = """select a.id 
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-11-30' 
              and a.posttypeid = {0} 
              and a.tags like '%<python>%' 
              and a.id in ({1})
"""

In [19]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql.format(q_postypeid, q_id_str))
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [20]:
q_output = pd.DataFrame(rows, columns = [
  'id'
  ,'title'
  , 'body'
])


In [21]:
q_output.head()

,id,title,body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett..."
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...
3,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...
4,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...


In [22]:
q_output = pd.merge(q_output, q_df[['id', 'Q_difficulty']], on = ['id'] )

In [23]:
def chg_tag(code):
    st_pattern = r'<pre(?: class="[^"]*")?><code>'
    st_dst = "```python\n"
    code = re.sub(st_pattern, st_dst, code, count=0, flags=0)
    
    end_dst = "```"
    end_pattern =r'</code></pre>'
    code = re.sub(end_pattern, end_dst, code, count=0, flags=0)
    return code
                                                

In [24]:
q_output['t_body'] = q_output['body'].apply(chg_tag)

In [25]:
q_output['clean_body'] = q_output['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))

In [26]:
q_output['clean_body'] = q_output['clean_body'].apply(lambda x:  re.sub(r";(?=\S)", "", x))
# .str.replace('";', "")

In [27]:
q_output['answer'] = 'Difficulty Level : '+q_output['Q_difficulty'].map(str)

In [28]:
q_output['question'] = """<Title>"""+q_output['title'].map(str)+"""</Title>. <Question>"""+q_output['clean_body'].map(str)+"""</Question> Let's think through the difficulty of question carefully, step by step. """


In [29]:
# q_output[['id', 'question', 'answer']].to_csv('../../data/q_output_code_y_241201.csv')

In [30]:
# q_output.iloc[[14, 25, 13, 21], :].to_csv('chk.csv')

In [31]:
q_output.head()

,id,title,body,Q_difficulty,t_body,clean_body,answer,question
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett...",Advanced,"<p>According to <a href=""https://keras.io/gett...","According to this keras documentation, pickle ...",Difficulty Level : Advanced,<Title>Why joblib is not recommended when savi...
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...,Basic,<p>I want to replace multiple strings in a lis...,I want to replace multiple strings in a list w...,Difficulty Level : Basic,<Title>replace multiple strings in a list with...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...,Intermediate,<p>I have this function that I want to test:</...,I have this function that I want to test:\n```...,Difficulty Level : Intermediate,<Title>pandas unit test AssertionError: DataFr...
3,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...,Intermediate,<p>I am trying to iterate through a folder and...,I am trying to iterate through a folder and de...,Difficulty Level : Intermediate,<Title>Removing all duplicate images with diff...
4,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...,Intermediate,<p>I need help for my school project. I am tak...,I need help for my school project. I am taking...,Difficulty Level : Intermediate,<Title>How do I extract a binary pattern from ...


In [32]:
q_output[['id', 'question', 'answer']].to_csv('./q_output_code_y.csv')